# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов, теорий заговоров, постов в помоечных телеграм каналах, текстов журналистов и СМИ с выразительным стилем). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')
model_path = 'sberbank-ai/rugpt3medium_based_on_gpt2'

tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


В виде датасета используем данные с токсичными и нейтральными комментариями, где для каждого текста будет задан токен стиля [neutral] или [toxic].

In [ ]:
import pandas as pd

data_path = '/content/drive/MyDrive/labeled.csv'
data = pd.read_csv(data_path)

data['label'] = data['toxic'].map({1: 'toxic', 0: 'neutral'})
data['input'] = data['label'].apply(lambda x: '[' + x + '] ') + data['comment']

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(data, shuffle=True, stratify=data['label'])
train_data = train_data['input'].values.tolist()
val_data = val_data['input'].values.tolist()

In [ ]:
train_path = 'odno_train_dataset.txt'
with open(train_path, "w") as f:
    f.write('\n'.join(train_data))
    
val_path = 'odno_val_dataset.txt'
with open(val_path, "w") as f:
    f.write('\n'.join(val_data))

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,
                            file_path=train_path,
                            block_size=64, 
                            overwrite_cache=True)

val_dataset = TextDataset(tokenizer=tokenizer,
                          file_path=val_path,
                          block_size=64, 
                          overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
len(train_dataset), len(val_dataset)

(8377, 2781)

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 4
epochs = 2

training_arguments = TrainingArguments(output_dir='./models/',
                                       per_device_train_batch_size=batch_size, 
                                       per_device_eval_batch_size=batch_size,  
                                       num_train_epochs=epochs,
                                       logging_dir='./logs',
                                       logging_steps=50,
                                       eval_steps=200,
                                       save_steps=1000,
                                       warmup_steps=500,
                                       report_to='none',
                                       evaluation_strategy='steps'
                                       )

trainer = Trainer(model=model,
                  train_dataset=train_dataset,
                  eval_dataset=val_dataset,
                  data_collator=data_collator, 
                  args=training_arguments,
                  optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5),None)
                  )

PyTorch: setting up devices


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 8377
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 4190


Step,Training Loss,Validation Loss
200,3.650000,3.762605
400,3.645300,3.762216
600,3.695400,3.746014
800,3.786400,3.742364
1000,3.680500,3.739798
1200,3.716900,3.735901
1400,3.766300,3.728274
1600,3.744700,3.728084
1800,3.734600,3.724246
2000,3.689600,3.721359


***** Running Evaluation *****
  Num examples = 2781
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2781
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2781
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2781
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2781
  Batch size = 4
Saving model checkpoint to ./models/checkpoint-1000
Configuration saved in ./models/checkpoint-1000/config.json
Model weights saved in ./models/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2781
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2781
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2781
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2781
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2781
  Batch size = 4
Saving model checkpoint to ./models/checkpoint-2000
Configuration saved in ./models/checkpoint-2000/config.json
Model weig

TrainOutput(global_step=4190, training_loss=3.632416707405327, metrics={'train_runtime': 4804.2058, 'train_samples_per_second': 3.487, 'train_steps_per_second': 0.872, 'total_flos': 1944931429515264.0, 'train_loss': 3.632416707405327, 'epoch': 2.0})

In [ ]:
model_checkpoint_path = '/content/models/checkpoint-4000'
trained_model = GPT2LMHeadModel.from_pretrained(model_checkpoint_path).to(device)

loading configuration file /content/models/checkpoint-4000/config.json
Model config GPT2Config {
  "_name_or_path": "sberbank-ai/rugpt3medium_based_on_gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 2048,
  "n_special": 0,
  "output_past": true,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
def generate_comment(mode_, temperature=1.0, top_k=15, top_p=1, beam_search=True):
  begin = '[toxic]' if mode_ == 'toxic' else '[neutral]'
  input_ids = tokenizer.encode(begin, return_tensors="pt",).to(device)
  out = trained_model.generate(input_ids, 
                               do_sample=True, 
                               temperature=temperature, 
                               top_k=top_k, 
                               top_p=top_p, 
                               beam_search=beam_search, 
                               max_length=50)
  generated_text = list(map(tokenizer.decode, out))[0]
  return generated_text.split('\n')[0]

Генерируем с различными параметрами:

In [ ]:
for _ in range(3):
  print(generate_comment('toxic'))
  print()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[toxic] Я в жопе, бляха муха.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[toxic] А почему это не может быть в одном из следующих постов? В чем прикол?

[toxic] А ты в курсе, что это не так?



In [ ]:
for _ in range(3):
  print(generate_comment('toxic', top_k=40, top_p=0.9, temperature=0.9))
  print()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[toxic] Не надо так говорить! Ты что, думаешь, я тут один такой?



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[toxic] Тупая и злая обезьяна.

[toxic] И так по кругу. Если ты хочешь что бы твоя башка работала на благо страны то тебе надо запретить употреблять алкоголь. Что бы ты не говорил, это будет лишь поводом к тому, что ты будешь пиздить, что твоя



In [ ]:
for _ in range(3):
  print(generate_comment('toxic', top_k=40, top_p=0.99, temperature=0.6))
  print()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[toxic] Ты не понимаешь. Это же хохлы. Они на самом деле считают что Путин и Ко - это они. И они не понимают что это не так.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[toxic] Нахуй ты мне это пишешь? Мне похуй.

[toxic] А я считаю, что если у тебя есть деньги, то ты можешь купить что хочешь. А вот если нет, то ты должен с этим смириться и идти работать.



In [ ]:
for _ in range(3):
  print(generate_comment('neutral', top_k=40, top_p=0.95, temperature=1))
  print()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[neutral] А у меня просто в голове от такой картинки аж мысли роятся. Это что-то с чем-то.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[neutral] Просто я не представляю как в России вообще можно что-то делать без гос поддержки. Если я уеду куда-то и буду что-то делать, как тогда будут вестися дела? Даже если я просто уеду в

[neutral] С каждым годом это всё больше и больше.



In [ ]:
for _ in range(3):
  print(generate_comment('neutral', top_k=100, temperature=1.01))
  print()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[neutral] Если Вы считаете, что эта сеть является одной из лучших, то Вы ошибаетесь. Это действительно компания с многомиллионным оборотом, но не более того.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[neutral] Спасибо. Не думал, что такие умные люди могут писать. И что в результате получается действительно гениально.

[neutral] Спасибо, попробую.



In [ ]:
print(generate_comment('neutral'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[neutral] Я так же на али заказывала. В общем и целом не плохо. Но доставка не такая долгая и не такая удобная, как в России. Да и не все товары в наличии (по крайней мере для России)


## Примеры сгенерированных текстов:

[toxic] Тупая и злая обезьяна.<br>
[toxic] Нахуй ты мне это пишешь? Мне похуй.<br>
[toxic] Ты не понимаешь. Это же хохлы. Они на самом деле считают что Путин и Ко - это они. И они не понимают что это не так.<br>
[toxic] А я считаю, что если у тебя есть деньги, то ты можешь купить что хочешь. А вот если нет, то ты должен с этим смириться и идти работать.<br><br>

[neutral] Просто я не представляю как в России вообще можно что-то делать без гос поддержки. Если я уеду куда-то и буду что-то делать, как тогда будут вестися дела? Даже если я просто уеду в<br>
[neutral] Спасибо. Не думал, что такие умные люди могут писать. И что в результате получается действительно гениально.<br>
[neutral] Если Вы считаете, что эта сеть является одной из лучших, то Вы ошибаетесь. Это действительно компания с многомиллионным оборотом, но не более того.<br>

### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

1) 
GPT-1 -Improving Language Understanding by Generative Pre-Training https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf<br>
GPT-2 - Language Models are Unsupervised Multitask Learners https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf<br>
GPT-3 - Language Models are Few-Shot Learners https://arxiv.org/pdf/2005.14165.pdf<br><br>

2)
За основу был взят датасет CommonCrawl dataset.<br>
Он был почищен с помощью алгоритма сравнения между текущим текстом и текстами высокого качества (по сути классификация на качественный и некачественный текст, где классом качественных текстов являлись тексты WebText, а некачественный - Common Crawl Corpus). После чего тексты с низкой вероятностью класса качественного текста удалялись из обучающего набора.<br>
Использована дедупликация текстов внутри документа и между различными документами (удалены тексты с большим пересечением с другими): для улучшения отложенной валидации и предотвращения переобучения, и уменьшения избыточности обучающей выборки.<br>
К основному датасету CommonCrawl были добавлены высококачественные датасеты: WebText dataset, English Wiki, internet-based books (Books1, Books2).